In [1]:
#first import needed libraries
import numpy as np # numpy arrays
import pandas as pd # dataframes and many many other things
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # work with json files
import requests #make requests to urls

import urllib #open urls

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

# Task 1
So, we're opening using library urllib desired wikipedia website, which has the target table. <br>
After that, we're transferring html file derived into beatiful soup file. <br>
The last this is we use prettify method to make the file readable.

In [2]:
url_wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html=urllib.request.urlopen(url_wiki).read()
wiki_soup=BeautifulSoup(wiki_html,'html.parser')
#print(wiki_soup.prettify())
#alternative way to read table from link and conver to dataframe 
#df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#df = df[0]
#df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

### I really don't know how html works, so I just looked at the whole text at first (currently commented, because it's very long)
When looking at the file I've done text 'ctrl + F' search, how the table is coded in html, I've searched for the text words that was previous to the table ("here are no rural FSAs in Toronto") so, below the text part we fould the name ( table class="wikitable sortable" )

In [3]:
soup_table = wiki_soup.find('table',{'class':'wikitable sortable'})
#so then, after we locate the table, we only take that relevant part, and find every text line by searching 'td'
soup_links=soup_table.findAll('td')
columns=['Postcode', 'Borough', 'Neighborhood']
postal_codes=pd.DataFrame(columns=columns)
temp_dict={}
for n,line in enumerate(soup_links):
#basically, every element in 'soup_links' is every cell in the table, and as we know there are 3 elements in every row, so we are configuring this algorithm to
#re-write a dictionary 'temp_dict' on every 3-rd iteration and append it to the created dataframe 'postal_codes'
    curr=line.get_text().strip('\n') # at this line - we're getting only relevant text without any html symbols and other things, also we're stripping the next line symbol
#this if is served for multi-purpose 1. if n%3==0 then the we've got the full temp_dict recorded that we need to record
# however if n==0, then we're at the start of the loop, and we don't have anything to write yet
# also it is given at the task that we need to delete all unassigned lists
    if (n%3==0) and (n!=0) and (temp_dict['Borough']!='Not assigned'):
# next if is to see when the postcode already had been recorded, so if the loc method with condition inside shows that it's empty - then we need to write a new line
        if postal_codes.loc[postal_codes['Postcode']==temp_dict['Postcode']].empty:
            postal_codes=postal_codes.append(temp_dict,ignore_index=True)
        else:
# if the postcode already had been recorded - then let's append things to it! (for convinience, i've put long condition as 'cond')
            cond=postal_codes['Postcode']==temp_dict['Postcode']
            postal_codes.loc[cond,'Neighborhood']=postal_codes.loc[cond,'Neighborhood'].astype(str)+', '+temp_dict['Neighborhood']
    temp_dict[columns[n%3]]=curr
    
# if a neighbourhood not assigned, then it must be replaced
postal_codes.loc[postal_codes['Neighborhood']=='Not assigned','Neighborhood']=postal_codes.loc[postal_codes['Neighborhood']=='Not assigned','Borough']
postal_codes.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [4]:
postal_codes.shape

(103, 3)

# Task 2
Be reading a list of providers on https://geocoder.readthedocs.io, <br>
I've initally planned to use CanadaPost instead of google for geocoder, I registired on their website for this purposes <br>
link on the geocoder description of available providers https://geocoder.readthedocs.io/providers/CanadaPost.html, however apparently it has no location data present <br>
so then, after trying almost every provider there is, I discovered that geocodefarm is by far the best one (FYI yandex and OSI is working however their coordinates are just very off)

In [5]:
!pip install geocoder
import geocoder # import geocoder
#key_canadapost='<9fbd63fd35fb658f : 76c37056ae1906eef9197d>'
loc_file_url='http://cocl.us/Geospatial_data'

In [6]:
import time
# initialize your variable to None

# define a funtion that will use the geocoder and loop until you get the coordinates, or until half a second passes (2 iterations with 0.25 wait time)
def geocoder_finder(timestamp):
    i=0
    lat_lng_coords = None
    for i in range(2):
        #g = geocoder.canadapost('{}, Toronto, Ontario'.format(timestamp)) commented because canadapost apparently not showing any geographical coordinates
        g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(timestamp))
        lat_lng_coords = g.latlng
        time.sleep(0.25) # sleep time had been introduced because a lot of API's are asynchronic - and also have a limit of many requests knocking on their door
        if lat_lng_coords is not None:
            return lat_lng_coords
    return None

#### This part is loading the backup data - in case our geocoder goes wrong

In [7]:
geo_file=pd.read_csv(loc_file_url)
geo_file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Finally, we start a cycle with postcodes and gather the latitudes

In [8]:
postal_codes['Latitude']='NA'
postal_codes['Longitude']='NA'
postal_codes['Source']='NA'
# apart from Latitude and Longitude for each column, we're adding a source - notion, where the geocoordinates had been taken from
# unfortunately, geofarm free provides only 250 requests per day for a free plan, so if I make 2 requests and have some error with this dataset - 
#and on some attempts I'm not able to get all of the data from there. Fortunately, we have another sourse of data available.
for i,st in enumerate(postal_codes['Postcode'].values):
    latlong=geocoder_finder(st)
    if latlong==None:
        latlong=geo_file.loc[geo_file['Postal Code']==st,['Latitude','Longitude']].values[0]
        postal_codes.loc[postal_codes['Postcode']==st,['Source']]='Provided file'
    else:
        postal_codes.loc[postal_codes['Postcode']==st,['Source']]='Geodecodefarm'
    postal_codes.loc[postal_codes['Postcode']==st,['Latitude','Longitude']]=latlong
    #print(latlong)

postal_codes.tail()

Status code Unknown from https://www.geocode.farm/v3/json/forward/: ERROR - HTTPSConnectionPool(host='www.geocode.farm', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://www.geocode.farm/v3/json/forward/: ERROR - HTTPSConnectionPool(host='www.geocode.farm', port=443): Read timed out. (read timeout=5.0)


,Postcode,Borough,Neighborhood,Latitude,Longitude,Source
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069,Provided file
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,Provided file
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.6627,-79.3216,Provided file
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.6363,-79.4985,Provided file
102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.6288,-79.521,Provided file


In [9]:
#I'm saving this file as csv on local computer in order to have the step 2 end-result ready without running the whole programm
postal_codes.to_csv(r'Toronto_postcodes.csv')

# Task 3
Here we are opening the prepared csv file, enrigching it with foursquare data and run the KNN clustering model

In [2]:
#first we read some data
postal_codes=pd.read_csv('Toronto_postcodes.csv')
postal_codes.drop('Unnamed: 0',axis=1,inplace=True)
postal_codes.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Source
0,M3A,North York,Parkwoods,43.753259,-79.329656,Provided file
1,M4A,North York,Victoria Village,43.725882,-79.315572,Provided file
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Provided file
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Provided file
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,Provided file


In [3]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [4]:
# now let's filter the Boroughs to find only the ones related to Toronto
Toronto_postal=postal_codes.loc[postal_codes['Borough'].str.find('Toronto')!=-1]
Toronto_postal.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Source
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Provided file
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,Provided file
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Provided file
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,Provided file
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Provided file


In [5]:
#this part is for foursquare API
CLIENT_ID = 'FICA04YHTU00Y1DHOH1T5VALLE2VGNRGTJJBVB0ECZ4QEE1C' # your Foursquare ID
CLIENT_SECRET = 'LFO0GLMBQQQTCB1TCBSDFMKLRDGGEQK4O1VW5YLN4SR1NPHK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
#we need to define a function, that will gather all the needed nearby places in the neighborhood selected
def getNearbyVenues(names, latitudes, longitudes,limit=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url='https://api.foursquare.com/v2/venues/explore'

        params={
            'client_id':CLIENT_ID,
            'client_secret':CLIENT_SECRET,
            'v':VERSION,
            'll':str(lat)+','+str(lng),
            'limit':limit,
            'radius':radius
        }
        # make the GET request
        results = requests.get(url,params=params).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#so now we just need to apply created function of our desired list of neighborhoods
Toronto_venues=getNearbyVenues(Toronto_postal['Neighborhood'],Toronto_postal['Latitude'], Toronto_postal['Longitude'],radius=1000)
Toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3097,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Leslieville Flea Market,43.662779,-79.330250,Flea Market
3098,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Booty Camp Fitness,43.659976,-79.311994,Gym / Fitness Center
3099,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Breakfast Club,43.662811,-79.310174,Breakfast Spot
3100,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Carters Landing,43.662414,-79.309898,Bistro
3101,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Ashbridges Bay Yacht Club (ABYC),43.658210,-79.311004,Harbor / Marina


In [8]:
# on the previous cell radius had been increased because a good part of the Neighborhoods had less than 5 Venues identified
# and a lot of neighborhoods didn't even get at least one venue (only 38 out of 102)- now it's somewhat manageable
Toronto_result_stat=Toronto_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count().sort_values('Venue')
print(Toronto_result_stat.shape)
# as you can see, out from every identified neighborhood, we've found that only 38, even with expanding our radius search
Toronto_result_stat.head()

(38, 1)


,Venue
Neighborhood,
Lawrence Park,11
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15
Roselawn,25
Rosedale,27
"Cabbagetown, St. James Town",39


#### Now we need to perform so called "one-hot encoding"
We basically create dummies for each and every venue category - and then sum it in a normalized way (mean instead of count) for each neighborhood

In [9]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# move neighborhood column to the first column
Toronto_onehot['Neighborhood']=Toronto_venues['Neighborhood']
#I'm checking the shape just to make sure we didn't lose anything
print(Toronto_onehot.shape)
Cols=Toronto_onehot.columns.to_list()
Pos=Cols.index('Neighborhood')
Cols=[Cols[Pos]]+Cols[:Pos]+Cols[Pos+1:]
Toronto_onehot=Toronto_onehot[Cols]
Toronto_onehot.head()

(3102, 278)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped.head()

(38, 278)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.030000,0.0,0.0,0.0,0.0,0.01,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0
1,Berczy Park,0.0,0.0,0.000000,0.010000,0.0,0.0,0.0,0.0,0.01,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.000000,0.010000,0.0,0.0,0.0,0.0,0.01,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.020833,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.066667,0.000000,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


#### Now that we got the datasets, we cluster results

In [11]:
kclusters=3
Toronto_grouped_clustering=Toronto_grouped.drop('Neighborhood', 1)
#drop the neighborhood column, because we don't need splitting for clustering purposes
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

In [12]:
# first, we need to create a dataframe with Neighborhood, Borough, latitude and longitude for everything that had a venue
Result_table=Toronto_result_stat.drop('Venue',axis=1).join(postal_codes.drop('Source',axis=1).set_index('Neighborhood'))
Result_table.sort_values('Neighborhood',inplace=True)
Result_table.reset_index(drop=False, inplace=True)
#now we're inserting labels we got from the clustering algorithm
Result_table.insert(0, 'KNN Labels', kmeans.labels_)
print(Result_table.shape)
Result_table.head()

(38, 6)


,KNN Labels,Neighborhood,Postcode,Borough,Latitude,Longitude
0,0,"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568
1,0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
2,0,"Brockton, Exhibition Place, Parkdale Village",M6K,West Toronto,43.636847,-79.428191
3,1,Business Reply Mail Processing Centre 969 Eastern,M7Y,East Toronto,43.662744,-79.321558
4,1,"CN Tower, Bathurst Quay, Island airport, Harbo...",M5V,Downtown Toronto,43.628947,-79.394420


In [13]:
Result_table[['KNN Labels','Postcode']].groupby('KNN Labels').count()
#well, this isn't ideal, any number cluster bigger of smaller gives a lot more wanky division, so the 3 is the magic number

,Postcode
KNN Labels,
0,12
1,25
2,1


In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
Toronto_ll=[43.6532,-79.3832]
map_clusters = folium.Map(location=Toronto_ll, zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Result_table['Latitude'], Result_table['Longitude'], Result_table['Neighborhood'], Result_table['KNN Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Unfortunately, folium map is not transferred correctly into Github, so I'm inserting this picture for you to see the actual result
I encourage you to re-test my code and create that map inside the environment you're working on
![](https://raw.githubusercontent.com/Lovecraft-hp/Data_science_pile/master/Folium_snapshot.PNG)

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,American Restaurant
1,0,Berczy Park,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar
2,0,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant
3,1,Business Reply Mail Processing Centre 969 Eastern,Park,Brewery,Coffee Shop,Pizza Place,Burrito Place
4,1,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Coffee Shop,Café,Scenic Lookout,Track


## Cluster 1
first cluster has a lot more places to eat and drink, like coffee shops cafes, and various other entertainment venues, I'm guessing that's the center of the city (or should i call it downtown area?)

In [27]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0].drop('Cluster Labels',axis=1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,American Restaurant
1,Berczy Park,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Mexican Restaurant
10,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Steakhouse,Restaurant
14,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Steakhouse,Concert Hall
16,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Japanese Restaurant,Steakhouse
18,"Harbord, University of Toronto",Café,Bakery,Vegetarian / Vegan Restaurant,Bar,Restaurant
19,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Café,Italian Restaurant,Aquarium
23,"Little Portugal, Trinity",Café,Bar,Bakery,Coffee Shop,Cocktail Bar


## Cluster 2
Second place has more parks, shops and grocery stores, I'm guessing that's more an area with apartment blocks and houses

In [28]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1].drop('Cluster Labels',axis=1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Business Reply Mail Processing Centre 969 Eastern,Park,Brewery,Coffee Shop,Pizza Place,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Coffee Shop,Café,Scenic Lookout,Track
5,"Cabbagetown, St. James Town",Park,Japanese Restaurant,Gastropub,Diner,Coffee Shop
6,Central Bay Street,Coffee Shop,Ice Cream Shop,Café,Vegetarian / Vegan Restaurant,Ramen Restaurant
8,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Café,Sushi Restaurant
11,Davisville,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Dessert Shop
12,Davisville North,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Dessert Shop,Pizza Place
13,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Sushi Restaurant,Park,Italian Restaurant,Pizza Place
15,"Dovercourt Village, Dufferin",Café,Park,Coffee Shop,Bar,Sushi Restaurant


## Cluster 3
Not enough items to tell what this cluster is about

In [30]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2].drop('Cluster Labels',axis=1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Lawrence Park,Café,Pharmacy,College Quad,College Gym,Gym / Fitness Center
